In [1]:
%load_ext autoreload
%autoreload 2
import sys, warnings, time, numpy, yaml, pandas
sys.path.append("../src/") # go to parent dir
from data_access import get_X, get_y, get_train_test
from models.factory import ModelFactory
warnings.filterwarnings('ignore')

In [2]:
feature=['HOME_WINS', 'DRAW', 'AWAY_WINS']
train_data = get_X('train')
train_scores = get_y()
test_data = get_X('test')
X_train, y_train, X_test, y_test, target = get_train_test(train_size=0.8, random_state=42, feature=feature)

In [3]:
def eval_model(model, save_model=False):
    start = time.time()
    model.fit()
    score = model.evaluate(X_test)
    end = time.time()
    print(f'{model.name}={score} in {numpy.round((end-start), 2)}s')
    if save_model:
        model.save(test_data)
    return {'name': model.name, 'score': score, 'time': numpy.round((end-start), 2)}

In [4]:
%load_ext autoreload
%autoreload 2
name= 'random_forest'
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, train_scores)
model = factory.get_model(name)
line = eval_model(model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
random_forest=0.4791 in 13.41s


In [5]:
%load_ext autoreload
%autoreload 2
name= 'catboost'
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, train_scores)
model = factory.get_model(name)
line = eval_model(model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
catboost=0.4722 in 49.07s


In [6]:


score = model.evaluate(X_test)
print(f'{model.name}={score}')

catboost=0.4722


In [7]:
%load_ext autoreload
%autoreload 2
save_model = True
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, train_scores)
lines = []
for model in factory.get_models():
    model.fit()
    score = model.evaluate(X_test)
    lines.append(eval_model(model, save_model))
df = pandas.DataFrame(lines)
df = df.sort_values(by=['score'], ascending=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

xgb_gblinear=0.4945 in 9.27s
catboost=0.4722 in 52.13s
dummy=0.4401 in 0.03s
random_forest=0.4709 in 11.21s



77/77 [==============================] - 1s 6ms/step
keras_relu=0.4323 in 410.17s
793/793 [==============================] - 4s 5ms/step


In [8]:
df=df.sort_values(by=['score'], ascending=False)
print(df.sort_values(by=['score'], ascending=False))
# df.to_csv('../data/result.csv')

            name   score    time
0   xgb_gblinear  0.4945    9.27
1       catboost  0.4722   52.13
3  random_forest  0.4709   11.21
2          dummy  0.4401    0.03
4     keras_relu  0.4323  410.17
